In [ ]:
pip install scipy plyfile polyscope robust_laplacian

In [ ]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import NearestNeighbors

In [63]:
def diffuse(data,source,heat_amount,n_neighbors):
  '''
    KNN Heat Transfer
    Return a heated point cloud


    Parameters
    ----------
    data (pandas data frame): This is the input point cloud

    source (int): Index of the heat source point

    heat_amount (int): The amount of heat in source

    n_neighbors (int): The numer of neighbors in KNN algorithm  


    Returns
    -------
    Heated point cloud               

  '''


  nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(data)
  distances, indices = nbrs.kneighbors(data)
  data["Heat"] = 0
  processed = [source]
  data["Heat"][source] = heat_amount
  heat_amount = heat_amount - 1
  while (len(processed) < len(indices[:,0])) and (heat_amount > 0) :   #### may we want to add that NN is not empty
    if (len(indices[source,:].shape)) == 1:
       NN = indices[source,:][1:]
    else: 
      NN = indices[source,:][:,1:]
      NN = np.reshape(indices[source,:][:,1:],(indices[source,:][:,1:].shape[0]*indices[source,:][:,1:].shape[1],1))
      NN = NN.tolist() 
      NN = list(itertools.chain(*NN)) 
    for i in set(NN):
      if i in processed:
        NN.remove(i)
        pass
      else:  
        data["Heat"][i] = heat_amount
        processed.append(i)
    heat_amount = heat_amount - 1
    print("heat is: {}".format(heat_amount))
    source = list(set(NN))
  return (data,indices)

In [ ]:
!wget http://graphics.stanford.edu/pub/3Dscanrep/bunny.tar.gz

In [ ]:
!tar zxvf /content/bunny.tar.gz

In [48]:
import robust_laplacian
from plyfile import PlyData
import numpy as np
import polyscope as ps
import scipy.sparse.linalg as sla

# Read input
plydata = PlyData.read("/content/bunny/reconstruction/bun_zipper_res2.ply")
point = np.vstack((
    plydata['vertex']['x'],
    plydata['vertex']['y'],
    plydata['vertex']['z']
)).T

In [ ]:
### Use if you want to down sample your point cloud

# print(point.shape[0])
# a = np.random.randint(point.shape[0], size=int(point.shape[0]/.5))
# x = np.delete(point, a, axis=0)
# point = pd.DataFrame(x)
# print(point.shape[0])

In [ ]:
points = pd.DataFrame(point)
(data,indices) = diffuse(points,1,70,10)

In [ ]:
import plotly.express as px 
px.scatter_3d(x = data.iloc[:,0],  
                    y = data.iloc[:,1],  
                    z = data.iloc[:,2], 
                    color =  data["Heat"],  
                    # size='petal_length', 
                    # size_max = 20,  
                    opacity = 1)  